# Building RAG from Scratch (Open-source only!)


- Sentence Transformers as the embedding model
- Postgres as the vector store (we support many other [vector stores](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html) too!)
- Llama 2 as the LLM (through [llama.cpp](https://github.com/ggerganov/llama.cpp))

## Setup

We setup our open-source components.
1. Sentence Transformers
2. Llama 2
3. We initialize postgres and wrap it with our wrappers/abstractions.

#### Sentence Transformers

In [1]:
#%pip install llama-index-readers-file pymupdf
#%pip install llama-index-vector-stores-postgres
#%pip install llama-index-embeddings-huggingface
#%pip install llama-index-llms-llama-cpp

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Llama CPP

In this notebook, we use the [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) model, along with the proper prompt formatting.

Check out our [Llama CPP guide](https://gpt-index.readthedocs.io/en/stable/examples/llm/llama_2_llama_cpp.html) for full setup instructions/details.

In [3]:
#!pip install llama-cpp-python

In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
llama_model_loader: loaded meta data

#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [5]:
#!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [6]:
import psycopg2

# Define your database connection parameters
db_name = "rag_db"  # Use the existing database name here
host = "localhost"
password = "password"  # Replace with your actual PostgreSQL password
port = "5432"  # Default PostgreSQL port
user = "myuser"  # Replace with your actual PostgreSQL username

# Connect directly to the 'rag_db' database
conn = psycopg2.connect(
    dbname=db_name,  # Connect directly to 'rag_db'
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# Optionally, perform any operations on 'rag_db' using a cursor
with conn.cursor() as c:
    # Drop and create operations are not needed if the database already exists
    # Here you can perform other database setup actions if necessary
    print(f"Connected to {db_name} successfully.")

# Close the connection when done
#conn.close()

Connected to rag_db successfully.


In [7]:
from sqlalchemy import make_url

from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database="rag_db",  # Replace with your actual database name
    host="localhost",   # Adjust if your database is hosted elsewhere
    password="password",  # Replace with your actual password
    port="5432",        # Default PostgreSQL port
    user="myuser",      # Replace with your actual user name
    table_name="llama2_paper",  # Adjust if you want a different table name
    embed_dim=384,  # Ensure this matches your embedding dimension / openai embedding dimension
)

## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

We fast-track the steps here (can skip metadata extraction). More details can be found [in our dedicated ingestion guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/ingestion.html).

## Load Data: Data is in 'data' folder

In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [9]:
# Load all PDF documents from the folder and add metadata
folder_path = Path("./data")
documents = []

for pdf_file in folder_path.glob("*.pdf"):
    loader = PyMuPDFReader()
    loaded_docs = loader.load(file_path=str(pdf_file))
    print(f"{pdf_file.name}: {len(loaded_docs)} documents loaded")
    
    # Add metadata (e.g., filename) to each document
    for doc in loaded_docs:
        doc.metadata = {"source": pdf_file.name}
        documents.append(doc)

5.pdf: 10 documents loaded
4.pdf: 12 documents loaded
1.pdf: 18 documents loaded
3.pdf: 16 documents loaded
2.pdf: 10 documents loaded


In [10]:
print(documents[0])
print(len(documents))

Doc ID: da435705-d661-4144-ab9c-5bdfa9cfc16e
Text: JNS JOURNAL OF NUTRITIONAL SCIENCE RESEARCH ARTICLE Workers’
healthy eating practices during the COVID-19 pandemic and their
relationship with physical activity and quality of life Alana do
Nascimento Oliveira , Lize Stangarlin-Fiori and Caroline Opolski
Medeiros* Postgraduate Program in Food and Nutrition, Federal
University of Parana, Curitiba...
66


### 2. Use a Text Splitter to Split Documents

In [11]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [12]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    #print(f"Document {doc_idx} has {len(cur_text_chunks)} chunks")
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))   

In [13]:
print(len(text_chunks))

114


### 3. Manually Construct Nodes from Text Chunks

In [14]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
        metadata={
            **documents[doc_idxs[idx]].metadata,  # This now includes 'source'
            "doc_index": doc_idxs[idx]  # Optional: document index
        }
    )
    nodes.append(node)

In [15]:
nodes[1]

TextNode(id_='0e34ea13-da6b-4dee-93af-fc4d8bd4df7e', embedding=None, metadata={'source': '5.pdf', 'doc_index': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This change in the lifestyle of the population is partly, due to\nthe need for measures to contain the spread of COVID-19, such\nas social isolation. This has proven to be effective(8) and has led\nmany people to perform their work remotely.(9) This mobility\nrestriction has had direct effects on psychological factors, such\nas an increase in cases of anxiety and depression and a reduction\nin the practice of physical activities.(10–12) In addition, eating\nhabits were also inﬂuenced both by economic factors, due to the\nreduction in the population’s income, as well as by the\nconsumption of foods with higher energy density.(13,14)\nIn the period before the pandemic, the consumption of fresh\nand minimally processed foods represented approximately 70%\nof the total caloric intake by the

### 4. Generate Embeddings for each Node

Here we generate embeddings for each Node using a sentence_transformers model.

In [16]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="text")
    )
    node.embedding = node_embedding

In [17]:
print(type(nodes[0].embedding))
print(nodes[0].embedding)
print(len(nodes))

<class 'list'>
[-0.0033521901350468397, 0.023529022932052612, 0.031206179410219193, 0.011556013487279415, 0.04029195383191109, 0.057001177221536636, 0.019863374531269073, 0.002204005839303136, -0.001091738580726087, -0.008093681186437607, -0.018772142007946968, -0.06604456901550293, -0.0016525505343452096, 0.016629204154014587, -0.0016811549430713058, -0.029079772531986237, 0.007660300936549902, -0.024458175525069237, 0.0007334882975555956, 0.018140992149710655, -0.021003102883696556, -0.016628552228212357, -0.017473481595516205, -0.008948509581387043, 0.03527719900012016, 0.022544119507074356, -0.021890919655561447, -0.03420296311378479, -0.07296150177717209, -0.20341856777668, -0.021619297564029694, -0.04542139917612076, 0.0255585927516222, -0.005422128830105066, -0.06182954087853432, -0.0031680024694651365, 0.030594294890761375, 0.01684347353875637, -0.012634526006877422, -0.01120906975120306, 0.005322073586285114, 0.018391428515315056, 0.038591090589761734, -0.03580818325281143, -0

### 5. Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [18]:
vector_store.add(nodes)

['b5f2ede3-9ff7-4dc9-8c8d-171fb3aedd15',
 '0e34ea13-da6b-4dee-93af-fc4d8bd4df7e',
 '81208c36-9601-4301-8489-78be337f5b58',
 '80816c40-429e-4111-bb48-0dc66225b382',
 '831bc7d6-395c-422e-8d18-5c85a039c21a',
 '3a8009a0-5f34-44f7-8c38-1adf6d2a371c',
 'e426bfb4-7be0-4424-bb42-ddafb0348ed0',
 '98f372b0-6e28-4ea8-b902-d6c6b507c7cc',
 '17a5db21-8067-464a-9c96-f8744e750bdf',
 '2cc963f6-faae-4fc4-94c3-d96316179372',
 'c07627ef-afef-4686-8019-5eb37c4b1344',
 'dcb00c0f-0db8-43e3-b455-ae709e3d1838',
 'd8085c81-f48b-4eac-944b-f7f1a1fb433c',
 'a6ad18fd-1e4e-46d0-ba51-722544f75399',
 'aee81c17-0026-4cf9-831b-d020af561cb9',
 'cfa8856b-10b9-4e3b-a3d3-7fa2dd25f9a0',
 '2e0d2bbe-f239-4c06-849c-2e34805c1891',
 '250afb1c-70ed-4b20-bbe0-2d826166d6ff',
 '979c6930-349d-4a74-8243-126eddb4f41e',
 'bd6ab3a6-53d7-4747-b559-4490931ac818',
 'b1a808ab-50a5-49cc-b7cf-3a0e0b455a7b',
 '59f7b2d7-c3ba-41ca-a19f-d2d5fd0a07fb',
 '317981e8-4a78-4d96-a5f7-bec42a95d298',
 'ea5f8834-3486-4be4-98f9-c8d40f797bba',
 '3f93ed79-d42a-

## Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at our [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [24]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List
from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.vector_stores import VectorStoreQuery

In [25]:
class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [26]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

retriever

## Plug this into our RetrieverQueryEngine to synthesize a response

In [27]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [ ]:
query_str = "Which individuals play a central role in promoting healthy eating?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   10303.43 ms
llama_print_timings:      sample time =       1.00 ms /    39 runs   (    0.03 ms per token, 39117.35 tokens per second)
llama_print_timings: prompt eval time =   25975.76 ms /  1188 tokens (   21.87 ms per token,    45.73 tokens per second)
llama_print_timings:        eval time =    4286.65 ms /    38 runs   (  112.81 ms per token,     8.86 tokens per second)
llama_print_timings:       total time =   30278.27 ms /  1226 tokens


In [29]:
print(str(response))

 Professionals in the fields of health and food, such as dietitians, food scientists and technologists, play a central role in promoting healthy eating.


In [30]:
print(response.source_nodes[0].get_content())

Nutrients 2024, 16, 1365
2 of 18
reaching 16% of the adult population in 2022 [4]. At the same time, nowadays, cardiovascular
diseases are the main cause of death globally, representing 32% of all global deaths [5]. Parallel
and paradoxically, thinness has become progressively valued and fatphobia, the discrimina-
tion and stigmatization against fat individuals, has become a phenomenon widely present
worldwide [6–8]. Finally, food crises with important health, social and economic international
impacts, such as the “mad cow”, have also increased concerns about food consumption [9].
Attempts to define what constitutes a healthy diet and provide dietary recommenda-
tions have been made in different scientific fields and organizations, including the World
Health Organization (WHO). In response to the increase in prevalence rates of morbidity
and mortality associated with chronic non-communicable diseases, in 2004 the WHO ap-
proved the Global Strategy on Diet, Physical Activity and Health,

In [31]:
query_str = "What does nutrition literacy significantly predicted in the social-ecological framework,?"

response = query_engine.query(query_str)

Llama.generate: 13 prefix-match hit, remaining 505 prompt tokens to eval

llama_print_timings:        load time =   10303.43 ms
llama_print_timings:      sample time =       1.25 ms /    48 runs   (    0.03 ms per token, 38369.30 tokens per second)
llama_print_timings: prompt eval time =   10204.18 ms /   505 tokens (   20.21 ms per token,    49.49 tokens per second)
llama_print_timings:        eval time =    5005.76 ms /    47 runs   (  106.51 ms per token,     9.39 tokens per second)
llama_print_timings:       total time =   15225.90 ms /   552 tokens


In [32]:
print(str(response))

 Based on the provided context information, nutrition literacy significantly predicted in the social-ecological framework is "food and nutrition literacy (FNLIT) and its predictors in primary schoolchildren in Iran."


In [34]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='ce733f5f-7752-4811-884b-260ef917f811', embedding=None, metadata={'source': '4.pdf', 'doc_index': 20}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='; Omidvar, N.; Amini, M.; Abdollahi, M.; Eini-Zinab, H.; Amirhamidi, Z.;\nEsfandiari, S.; Nutbeam, D. Food and nutrition literacy (FNLIT) and its predictors in primary schoolchildren in Iran. Health\nPromot. Int. 2019, 34, 1002–1013. [CrossRef]\n40.\nLynn, M.R. Determination and quantiﬁcation of content validity. Nurs. Res. 1986, 35, 382–385. [CrossRef]\n41.\nNielsen-Bohlman, L.; Panzer, A.; Kindig, D. Health Literacy: A Prescription to End Confusion; National Academies Press: Washington,\nDC, USA, 2004.\n42.\nLiao, L.-L.; Lai, I.-J.; Shih, S.-F.; Chang, L.-C. Development and validation of the nutrition literacy measure for Taiwanese college\nstudents. Taiwan J. Public Health 2018, 37, 582–597.\n43.\nStory, M.; Neumark-Sztainer, D.; French, S. Individual and Environm

In [33]:
print(response.source_nodes[0].get_content())

; Omidvar, N.; Amini, M.; Abdollahi, M.; Eini-Zinab, H.; Amirhamidi, Z.;
Esfandiari, S.; Nutbeam, D. Food and nutrition literacy (FNLIT) and its predictors in primary schoolchildren in Iran. Health
Promot. Int. 2019, 34, 1002–1013. [CrossRef]
40.
Lynn, M.R. Determination and quantiﬁcation of content validity. Nurs. Res. 1986, 35, 382–385. [CrossRef]
41.
Nielsen-Bohlman, L.; Panzer, A.; Kindig, D. Health Literacy: A Prescription to End Confusion; National Academies Press: Washington,
DC, USA, 2004.
42.
Liao, L.-L.; Lai, I.-J.; Shih, S.-F.; Chang, L.-C. Development and validation of the nutrition literacy measure for Taiwanese college
students. Taiwan J. Public Health 2018, 37, 582–597.
43.
Story, M.; Neumark-Sztainer, D.; French, S. Individual and Environmental Inﬂuences on Adolescent Eating Behaviors. J. Am. Diet.
Assoc. 2002, 102, S40–S51. [CrossRef]
44.
Tabbakh, T.; Freeland-Graves, J. Development and validation of the Multidimensional Home Environment Scale (MHES) for
adolescents an